In [1]:
"""
Grid preparation for future aspen project:

Author: maxwell.cook@colorado.edu
"""

import os, sys, time, re
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
import rasterio as rio
import numpy as np

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/')

# Custom functions
sys.path.append(os.path.join(maindir,'aspen-fire/Aim2/code/Python'))
from __functions import *

proj = 'EPSG:5070' # albers

print("Ready to go !")

Ready to go !


## Future Fire

From Stephens et al., In review, annual probability of area burned and fire occurrence. Trend to 2060 was calculated using a median-based linear model (MBLM) thielsen estimate. 

In [2]:
fp = os.path.join(projdir,'Aim3/data/spatial/mod/future_fire_grid_trend.gpkg')
future_fire = gpd.read_file(fp)
future_fire.head()

,grid_id,trend_area,trend_count,p_area,p_count,NA_L3NAME,US_L4NAME,US_L4CODE,geometry
0,159230.0,190.629216,0.019221,3.590523e-11,1.436235e-13,Southern Rockies,Foothill Shrublands,21d,"POLYGON ((-861518.632 2246765.246, -858285.087..."
1,159231.0,190.629216,0.019221,3.590523e-11,1.436235e-13,Southern Rockies,Foothill Shrublands,21d,"POLYGON ((-858285.087 2246403.307, -855051.389..."
2,159232.0,190.629216,0.019221,3.590523e-11,1.436235e-13,Southern Rockies,Foothill Shrublands,21d,"POLYGON ((-855051.389 2246042.730, -851817.539..."
3,159233.0,301.282740,0.026862,1.926234e-07,1.677490e-09,Southern Rockies,Crystalline Mid-Elevation Forests,21c,"POLYGON ((-851817.539 2245683.513, -848583.540..."
4,159234.0,411.936265,0.034504,3.852110e-07,3.354837e-09,Southern Rockies,Crystalline Mid-Elevation Forests,21c,"POLYGON ((-848583.540 2245325.656, -845349.389..."


In [3]:
# check for duplicates, remove them
n = future_fire.duplicated(subset=['grid_id']).sum()
if n > 0:
    print(f"\nThere are [{n}] duplicate rows.\n")
else:
    print("\nNo duplicates at this stage.\n")


No duplicates at this stage.



### Wildland Urban Interface/Intermix (SILVIS)

In [4]:
# calculate the zonal stats for the WUI classes
# load the SILVIS Lab 10-m WUI classification
wui_fp = os.path.join(projdir, 'Aim3/data/spatial/raw/silvis/srm_wui_silvis_10m.tif')
wui = rxr.open_rasterio(wui_fp, masked=True, chunks='auto').squeeze()
print(wui)

<xarray.DataArray (y: 84754, x: 51708)> Size: 18GB
dask.array<getitem, shape=(84754, 51708), dtype=float32, chunksize=(2595, 51708), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
  * x            (x) float64 414kB -1.193e+06 -1.193e+06 ... -6.831e+05
  * y            (y) float64 678kB 2.253e+06 2.253e+06 ... 1.392e+06 1.392e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0


In [5]:
# calculate the percent cover of WUI classes for each grid
t0 = time.time()

# see __functions.py
wui_grid = compute_band_stats(future_fire, wui, 'grid_id', attr='wui')
# tidy columns in the summary table
wui_grid['count'] = wui_grid['count'].astype(int)
wui_grid['total_pixels'] = wui_grid['total_pixels'].astype(int)
wui_grid.rename(columns = {'count': 'wui_pixels'}, inplace=True)

print(wui_grid.head()) # check the results

t1 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t1:.2f} minutes.")
print("\n~~~~~~~~~~\n")

    grid_id  wui  wui_pixels  total_pixels  pct_cover
0  159230.0    0        2220        145909   1.521496
1  159230.0    3        9532        145909   6.532839
2  159230.0    4          43        145909   0.029470
3  159230.0    6      131622        145909  90.208281
4  159230.0    8        2492        145909   1.707914

Total elapsed time: 1.49 minutes.

~~~~~~~~~~



In [6]:
wui_grid['wui'].unique()

array([0, 3, 4, 6, 8, 5, 1, 2, 7])

In [7]:
# join the WUI description
# create the mappings
wui_desc = {
    0: 'No Data',
    1: 'Forest/Shrubland/Wetland-dominated Intermix WU',
    2: 'Forest/Shrubland/Wetland-dominated Interface WUI',
    3: 'Grassland-dominated Intermix WUI',
    4: 'Grassland -dominated Interface WUI',
    5: 'Non-WUI: Forest/Shrub/Wetland-dominated',
    6: 'Non-WUI: Grassland-dominated',
    7: 'Non-WUI: Urban',
    8: 'Non-WUI: Other'
}

# join back to the results
wui_grid['wui_desc'] = wui_grid['wui'].map(wui_desc)
wui_grid.head()

,grid_id,wui,wui_pixels,total_pixels,pct_cover,wui_desc
0,159230.0,0,2220,145909,1.521496,No Data
1,159230.0,3,9532,145909,6.532839,Grassland-dominated Intermix WUI
2,159230.0,4,43,145909,0.029470,Grassland -dominated Interface WUI
3,159230.0,6,131622,145909,90.208281,Non-WUI: Grassland-dominated
4,159230.0,8,2492,145909,1.707914,Non-WUI: Other


In [8]:
# save this file out
out_fp = os.path.join(projdir,'Aim3/data/tabular/srm_grid_wui_summary.csv')
wui_grid.to_csv(out_fp)
print(f"File saved to: {out_fp}")

File saved to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim3/data/tabular/srm_grid_wui_summary.csv


In [9]:
# pivot the table to get WUI percent cover
# retain columns for just the interface/intermix classes
wui_w = wui_grid.pivot_table(index='grid_id', columns='wui', values='pct_cover', fill_value=0)
wui_w = wui_w[[1, 2, 3, 4]] # keep only WUI classes
wui_w.columns = [f"wui{int(col)}" for col in wui_w.columns] # rename the columns
wui_w = wui_w.reset_index()
wui_w.head()

,grid_id,wui1,wui2,wui3,wui4
0,159230.0,0.0,0.0,6.532839,0.029470
1,159231.0,0.0,0.0,15.458371,0.006854
2,159232.0,0.0,0.0,35.265412,0.000000
3,159233.0,0.0,0.0,11.896978,0.000000
4,159234.0,0.0,0.0,12.335940,0.000000


In [11]:
# reclassify into a binary raster (WUI / Non-WUI)
wui_bin = xr.where(wui.isin([1,2,3,4]), 1, 0)
wui_bin = wui_bin.assign_attrs(wui.attrs) # assign the array attributes
# save this raster out
out_path = os.path.join(projdir, 'Aim3/data/spatial/mod/silvis/srm_wui_silvis_10m_bin.tif')
wui_bin.rio.to_raster(
    out_path,
    compress='zstd', zstd_level=9,
    dtype=rio.uint8, driver='GTiff'
)
print(f"Saved file to: {out_path}")

Saved file to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim3/data/spatial/mod/silvis/srm_wui_silvis_10m_bin.tif


In [13]:
# resample to a coarser resolution to compute the euclidean distance

import subprocess
from osgeo import gdal

# define a function for the gdal warp implementation
def resample_grid_gdalwarp(in_file, out_file, extent=None, resample_method='max', res=250):
    """
    Resample a raster file from 10-meter to 250-meter resolution using gdalwarp.

    :param in_file: The input raster file path.
    :param out_file: The output raster file path.
    :param resampling_method: The resampling method to use ('sum', 'average', etc.).
    :param target_resolution: The target resolution in meters (default is 250).
    :return: None
    """
    try:
        # Construct the gdalwarp command
        command = [
            'gdalwarp',
            '-tr', str(res), str(res),  # Target resolution
            '-r', resample_method,  # Resampling method
            '-of', 'GTiff',  # Output format
            '-co', 'COMPRESS=LZW',  # Compression
            '-ot', 'Int8', # ensure output data type
            '-t_srs', 'EPSG:5070', # output CRS
            in_file,
            out_file
        ]

        # Run the command
        subprocess.run(command, check=True)
        print(f'Successfully resampled {in_file} to {out_file}')

    except subprocess.CalledProcessError as e:
        print(f"Error resampling {in_file}: {e}")

# apply the function
in_path = os.path.join(projdir, 'Aim3/data/spatial/mod/silvis/srm_wui_silvis_10m_bin.tif')
out_file = os.path.join(projdir, 'Aim3/data/spatial/mod/silvis/srm_wui_silvis_10m_bin_250m.tif')
resample_grid_gdalwarp(in_path, out_file) # run the process

Creating output file that is 2040P x 3447L.
Processing /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim3/data/spatial/mod/silvis/srm_wui_silvis_10m_bin.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Successfully resampled /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim3/data/spatial/mod/silvis/srm_wui_silvis_10m_bin.tif to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim3/data/spatial/mod/silvis/srm_wui_silvis_10m_bin_250m.tif


In [15]:
# calculate the euclidean distance array
from scipy.ndimage import distance_transform_edt
# load the 250m resampled grid created above
wui_250 = rxr.open_rasterio(out_file, masked=True).squeeze()

t0 = time.time()
# run the euclidean distance
distance_to_wui = distance_transform_edt(wui_250 == 0) * 250
# convert to an xarray data array
distance_xr = xr.DataArray(distance_to_wui, coords=wui_250.coords, dims=wui_250.dims, attrs=wui_250.attrs)

# export the result
distance_fp = os.path.join(projdir, 'Aim3/data/spatial/mod/silvis/distance_to_wui.tif')
distance_xr.rio.to_raster(
    distance_fp,
    compress="zstd", zstd_level=9,  # Efficient compression
    dtype="float32", driver="GTiff"
)

t1 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t1:.2f} minutes.")
print("\n~~~~~~~~~~\n")


Total elapsed time: 0.01 minutes.

~~~~~~~~~~



In [16]:
# calculate the average distance to WUI for gridcells

t0 = time.time()

# run the zonal stats
zs = compute_band_stats(
    geoms=future_fire, 
    image_da=distance_xr, 
    id_col='grid_id', 
    stats=['mean'], # 'median','std','percentile_90'
    attr='wui_dist',
    ztype='continuous'
)

# check the results:
print(zs.head())

t1 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t1:.2f} minutes.")
print("\n~~~~~~~~~~\n")

    grid_id  wui_dist_mean
0  159230.0    1038.492751
1  159231.0     636.598957
2  159232.0     308.430041
3  159233.0     721.659559
4  159234.0    1341.741558

Total elapsed time: 0.09 minutes.

~~~~~~~~~~



In [18]:
# merge the WUI summary stats by gridcell
wui_grid_stats = pd.merge(wui_w, zs, on='grid_id', how='right')
wui_grid_stats.head()

,grid_id,wui1,wui2,wui3,wui4,wui_dist_mean
0,159230.0,0.0,0.0,6.532839,0.029470,1038.492751
1,159231.0,0.0,0.0,15.458371,0.006854,636.598957
2,159232.0,0.0,0.0,35.265412,0.000000,308.430041
3,159233.0,0.0,0.0,11.896978,0.000000,721.659559
4,159234.0,0.0,0.0,12.335940,0.000000,1341.741558


In [20]:
print(len(future_fire))
print(len(wui_grid_stats))

10319
10319


In [22]:
wui_grid_stats.isna().sum()

grid_id          0
wui1             0
wui2             0
wui3             0
wui4             0
wui_dist_mean    0
dtype: int64

In [24]:
# export the summary table
out_fp = os.path.join(projdir, 'Aim3/data/tabular/srm_wui_grid_stats.csv')
wui_grid_stats.to_csv(out_fp)
print(f"Saved to: {out_fp}")

Saved to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim3/data/tabular/srm_wui_grid_stats.csv
